In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
X = np.random.rand(100, 2)  # 100 mẫu, 2 đặc trưng
y = np.random.randint(0, 2, 100)  # nhãn nhị phân


In [3]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
# Chuyển đổi dữ liệu thành tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [6]:
# Mô hình với 8 nút trong lớp ẩn
class Model_8(nn.Module):
    def __init__(self):
        super(Model_8, self).__init__()
        self.fc1 = nn.Linear(2, 8)  # Lớp ẩn 1 với 8 nút
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(8, 1)  # Đầu ra với 1 nút (Sigmoid)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


In [7]:
# Khởi tạo mô hình, loss function và optimizer
model_8 = Model_8()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_8.parameters(), lr=0.001)

In [8]:
# Huấn luyện mô hình
epochs = 100
for epoch in range(epochs):
    model_8.train()
    optimizer.zero_grad()
    output = model_8(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # In mỗi 10 epochs
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")


Epoch [0/100], Loss: 0.6813
Epoch [10/100], Loss: 0.6793
Epoch [20/100], Loss: 0.6776
Epoch [30/100], Loss: 0.6758
Epoch [40/100], Loss: 0.6741
Epoch [50/100], Loss: 0.6725
Epoch [60/100], Loss: 0.6708
Epoch [70/100], Loss: 0.6693
Epoch [80/100], Loss: 0.6677
Epoch [90/100], Loss: 0.6663


In [9]:
# Đánh giá mô hình trên tập kiểm tra
model_8.eval()
with torch.no_grad():
    y_pred = model_8(X_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Loss cuối cùng với 8 nút: {loss.item():.4f}")
    print(f"Accuracy với 8 nút: {accuracy.item():.4f}")

Loss cuối cùng với 8 nút: 0.6651
Accuracy với 8 nút: 0.3500


In [14]:
# Mô hình với 2 lớp ẩn (8 nút và 6 nút)
class Model_8_6(nn.Module):
    def __init__(self):
        super(Model_8_6, self).__init__()
        self.fc1 = nn.Linear(2, 8)  # Lớp ẩn 1 với 8 nút
        self.fc2 = nn.Linear(8, 6)  # Lớp ẩn 2 với 6 nút
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(6, 1)  # Đầu ra với 1 nút (Sigmoid)

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Lớp ẩn 1 với ReLU
        x = self.relu(self.fc2(x))  # Lớp ẩn 2 với ReLU
        x = torch.sigmoid(self.fc3(x))  # Đầu ra với Sigmoid
        return x


In [15]:
# Khởi tạo mô hình, loss function và optimizer
model_8_6 = Model_8_6()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_8_6.parameters(), lr=0.001)


In [16]:
# Huấn luyện mô hình
epochs = 100
for epoch in range(epochs):
    model_8_6.train()
    optimizer.zero_grad()
    output = model_8_6(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # In mỗi 10 epochs
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")


Epoch [0/100], Loss: 0.6944
Epoch [10/100], Loss: 0.6909
Epoch [20/100], Loss: 0.6877
Epoch [30/100], Loss: 0.6844
Epoch [40/100], Loss: 0.6813
Epoch [50/100], Loss: 0.6787
Epoch [60/100], Loss: 0.6763
Epoch [70/100], Loss: 0.6742
Epoch [80/100], Loss: 0.6722
Epoch [90/100], Loss: 0.6703


In [17]:
# Đánh giá mô hình trên tập kiểm tra
model_8_6.eval()
with torch.no_grad():
    y_pred = model_8_6(X_test_tensor)
    y_pred = (y_pred > 0.5).float()  # Chuyển đầu ra về nhị phân (0 hoặc 1)
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Loss cuối cùng với 8+6 nút: {loss.item():.4f}")
    print(f"Accuracy với 8+6 nút: {accuracy.item():.4f}")

Loss cuối cùng với 8+6 nút: 0.6685
Accuracy với 8+6 nút: 0.4000


## So sánh kết quả của 3 mô hình (4 nút, 8 nút, 8+6 nút):
***1. Mất mát cuối cùng thay đổi thế nào?***

### Mô hình 4 nút:

**Khi chỉ có 4 nút trong lớp ẩn, mô hình có ít khả năng học các đặc trưng phức tạp của dữ liệu. Do đó, giá trị mất mát cuối cùng có thể khá cao, vì mô hình chưa đủ mạnh để phân loại chính xác các mẫu trong dữ liệu.**

### Mô hình 8 nút:

**Khi số nút trong lớp ẩn được tăng lên 8, mô hình có khả năng học tốt hơn và thể hiện sự giảm mất mát so với mô hình 4 nút. Việc tăng số nút cho phép mô hình học các đặc trưng phức tạp hơn của dữ liệu, giúp giảm thiểu sự sai lệch trong dự đoán.**

### Mô hình 8+6 nút:

**Khi thêm một lớp ẩn thứ hai với 6 nút, mô hình sẽ có thêm khả năng học và kết hợp các đặc trưng ở các cấp độ sâu hơn. Mất mát cuối cùng có thể tiếp tục giảm, nhưng sự giảm này sẽ ít đáng kể nếu dữ liệu không đủ phức tạp để mô hình tận dụng thêm lớp ẩn thứ hai. Mặc dù vậy, thêm lớp ẩn thứ hai vẫn giúp mô hình có khả năng học sâu hơn.**

***2. Độ chính xác có cải thiện không? Tại sao bạn nghĩ vậy?***

### Mô hình 4 nút:

**Độ chính xác sẽ tương đối thấp khi chỉ có 4 nút trong lớp ẩn, vì mô hình không có đủ độ phức tạp để học các mối quan hệ giữa các đặc trưng trong dữ liệu. Do đó, mô hình này có thể dễ dàng bị thiếu sót khi phân loại đúng các mẫu trong dữ liệu kiểm tra.**

### Mô hình 8 nút:

**Khi tăng số lượng nút lên 8, độ chính xác có thể cải thiện do mô hình có khả năng học các đặc trưng phức tạp hơn. Tuy nhiên, nếu mô hình không được huấn luyện đủ kỹ hoặc dữ liệu không đủ phức tạp, độ chính xác có thể không cải thiện đáng kể so với mô hình 4 nút.**

### Mô hình 8+6 nút:

**Độ chính xác có thể cải thiện khi thêm một lớp ẩn thứ hai với 6 nút, nhưng mức độ cải thiện sẽ phụ thuộc vào khả năng mô hình tổng hợp và học các đặc trưng phức tạp từ dữ liệu. Trong một số trường hợp, việc tăng số lượng lớp và nút có thể dẫn đến overfitting (quá khớp), làm giảm độ chính xác trên tập kiểm tra, đặc biệt nếu mô hình học quá nhiều chi tiết không cần thiết.**

***3. Tại sao độ chính xác có thể cải thiện?***

**Sự cải thiện độ chính xác xảy ra khi mô hình có khả năng học thêm các đặc trưng của dữ liệu. Khi số lượng nút trong lớp ẩn được tăng lên, mô hình có thể học các mối quan hệ phức tạp hơn giữa các đặc trưng và nhãn mục tiêu, từ đó đưa ra dự đoán chính xác hơn.**

***Lý do không cải thiện:***

**Mặc dù mô hình có thêm lớp ẩn hoặc số nút trong lớp ẩn, nếu dữ liệu không đủ phức tạp hoặc quá ít mẫu để mô hình học tốt, độ chính xác có thể không cải thiện. Hơn nữa, khi mô hình trở nên phức tạp hơn, nguy cơ overfitting cũng tăng lên, làm giảm hiệu suất trên tập kiểm tra.**